# Graphical Models for Biological Data : Final Project 

- Fall 2024 
- Authors: jtm98, Ned Ed Neddy 

# Model 0: Naive Estimator 

$$ Y_i \sim X_i * \beta $$

$$\beta \sim X_{experiment} / X_{control} $$


Naively, one can estimate $\beta$ for a gene-enhancer by dividing the average experiment expression by the average control expression. 


In [14]:
import os
import numpy as np 
from cmdstanpy import cmdstan_path, CmdStanModel

# Model 1: Massaging Gaussian Mixture Model 

jtm98 
Created: 10.29.24

$$ X_i \sim r N (\mu,\sigma^2) + (1-r) N (\beta*\mu,\sigma^2) $$

- $X_i$ is cell expression 
- $r$ is a mixture proportion of label $Z_i$
- $Z_i \in {0,1} $ and represents whether a cell had normal (0) or decreased (1)  expression 
- $\mu$ is the mean of individuals w/normal expression
- $\beta$ is the enhancer coefficient 


In [ ]:
mixture ="""

// Assessing if beta parameter is inferable 
data {
int<lower=1> N; // number of samples
array[N] real X; // expression in the sample 
}

parameters {
real<lower=0,upper=1> r; // mixture proportion of samples 
real<lower=0.0> sigma; // standard deviation 
real mu; // mean of individuals w/ normal expression
real beta; // enhancer coefficient
}

model {
r ~ beta(2,2); // mixture proportion based on natural scale 
mu ~ normal(0,10); 
beta ~ beta(2,2); // enhancer coefificent should be [0,1]
sigma ~ inv_gamma(1,1); 
for(i in 1:N)
target+= log_sum_exp(log(r)+normal_lpdf(X[i]|mu,sigma),
log(1-r)+normal_lpdf(X[i]|beta*mu,sigma));
}

generated quantities {
array[N] real<lower=0,upper=1> PZi;
for(i in 1:N) {
real LP0=log(r)+normal_lpdf(X[i]|mu,sigma);
real LP1=log(1-r)+normal_lpdf(X[i]|beta*mu,sigma);
PZi[i]=exp(LP0-log_sum_exp(LP0,LP1));
}
}
"""
! mkdir mixture
with open('mixture/mixture.stan', 'w') as f: f.write(mixture) 
! cat mixture/mixture.stan

In [3]:
mixture = CmdStanModel(stan_file='mixture/mixture.stan')

21:55:57 - cmdstanpy - INFO - compiling stan file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture.stan to exe file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture
21:56:06 - cmdstanpy - INFO - compiled model executable: /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/mixture/mixture


In [5]:

pos_expression = 100
neg_expression = 90 
true_beta = neg_expression / pos_expression # true beta value

# Simulate random data 
pos = np.random.normal(100,0.3,50)
neg = np.random.normal(90,0.3,50)
data = {'X' : np.concatenate([pos,neg]), 'N' : 100}
inits = { 'mu': 1, 'sigma': 0.3, 'r': 0.5, 'beta': .5 }

In [ ]:
mixture_fit = mixture.sample(data=data,thin=1,iter_sampling=1000,iter_warmup=50,inits=inits,seed=11051999)
mixture_fit.summary()['Mean']

In [8]:
estimated_beta = mixture_fit.summary().loc['beta']['Mean']
print('Estimated Beta:',estimated_beta)
print('True Beta:',true_beta)

Estimated Beta: 0.709097
True Beta: 0.9


# Model 2: Hierarchical Mixture Model

Updated: 11.09.24


$$ X_i \sim r N (\mu,\sigma^2) + (1-r) N (\beta_{enhancer}*\mu,\sigma^2) $$

- $X_i$ is cell expression 
- $r$ is a mixture proportion of label $Z_i$
- $Z_i \in {0,1} $ and represents whether a cell had non-functional (0) or functional (1) guide 
- $\mu$ is the mean of individuals w/normal expression
- $\beta$ is the enhancer coefficient 

In [15]:
# Model 0: Naive Estimator with no guide effect 
# Model 1: Mixture 
# Model 2: Mixture with guide potency and empirical priors 
# Model 3: Mixture with guide functionality generated quantities 
# Model 4: Pooled mixtured 

stan_code ="""
data {
  int<lower=0> N_control;                   // Number of control samples
  int<lower=0> N_experiment;                // Number of experimental samples
  int<lower=0> N_enhancers;                 // Number of enhancers
  int<lower=0> N_guides;                     // Number of guides
  array[N_control] real<lower=0> y_control;  // Expression levels in control samples
  array[N_experiment] real<lower=0> y_experiment;   // Expression levels in experimental samples
  array[N_experiment] int<lower=1> guide;  // Guide index for each experiment sample (1 to 5)
  array[N_experiment] int<lower=1> enhancer; // Enhancer index for each experiment sample (1 to 5)
}

transformed data {
  real<lower=0> mu_control_observed = mean(y_control);            // Empirical mean of y_control
  real<lower=0> sigma_control_observed = sd(y_control);           // Empirical standard deviation of y_control
  
  real<lower=0> mu_experiment_observed = mean(y_experiment);      // Empirical mean of y_experiment
  real<lower=0> sigma_experiment_observed = sd(y_experiment);     // Empirical standard deviation of y_experiment

  real beta_observed = mu_experiment_observed / mu_control_observed;   // Empirical effect size 
}

parameters {
  real<lower=0> mu_control;                 // Mean expression in control condition
  array[N_enhancers] real<lower=0, upper=1> B;        // Effect sizes for each enhancer
  real<lower=0, upper=10000> sigma_control; // Standard deviation in control condition
  real<lower=0, upper=10000> sigma_experiment; // Standard deviation in experiment condition
  array[N_guides] real<lower=0, upper=1> potency;  // Potency for each guide (0 to 1)
}

transformed parameters {
  array[N_enhancers] real<lower=0> mu_experiment;     // Expression levels for each enhancer

  for (e in 1:N_enhancers) {
    mu_experiment[e] = B[e] * mu_control;   // Calculate experiment mean for each enhancer
  }
}

model {
  // Priors based on provided initial values
  mu_control ~ normal(mu_control_observed, sigma_control_observed);
  B ~ beta(0.5, 0.5);                      // Priors for each enhancer effect size
  sigma_control ~ cauchy(sigma_control_observed, 2);
  sigma_experiment ~ cauchy(sigma_experiment_observed, 2);
  potency ~ beta(0.5, 0.5);                // Prior for potency of each guide

  // Likelihoods
  y_control ~ normal(mu_control, sigma_control); // Control samples from control distribution

  for (n in 1:N_experiment) {
    real mean_expr = (1 - potency[guide[n]]) * mu_control + potency[guide[n]] * mu_experiment[enhancer[n]];
    y_experiment[n] ~ normal(mean_expr, sigma_experiment);
  }
}

/*
generated quantities {
  array[N_guides] int<lower=0, upper=1> guide_functionality;

  for (g in 1:N_guides) {
    real log_nonfunctional = 0;
    real log_functional = 0;

    for (n in 1:N_experiment) {
      if (guide[n] == g) {
        log_nonfunctional += log1m(potency[g]) + normal_lpdf(y_experiment[n] | mu_control, sigma_control);
        log_functional += log(potency[g]) + normal_lpdf(y_experiment[n] | mu_experiment[enhancer[n]], sigma_experiment);
      }
    }
    guide_functionality[g] = log_functional > log_nonfunctional ? 1 : 0;
  }
}
*/

"""
name = 'model4_pooled'
! mkdir {name}
with open('%s/%s.stan'%(name,name), 'w') as f: f.write(stan_code) 
! cat {name}/{name}.stan

mkdir: model4_pooled: File exists

data {
  int<lower=0> N_control;                   // Number of control samples
  int<lower=0> N_experiment;                // Number of experimental samples
  int<lower=0> N_enhancers;                 // Number of enhancers
  int<lower=0> N_guides;                     // Number of guides
  array[N_control] real<lower=0> y_control;  // Expression levels in control samples
  array[N_experiment] real<lower=0> y_experiment;   // Expression levels in experimental samples
  array[N_experiment] int<lower=1> guide;  // Guide index for each experiment sample (1 to 5)
  array[N_experiment] int<lower=1> enhancer; // Enhancer index for each experiment sample (1 to 5)
}

transformed data {
  real<lower=0> mu_control_observed = mean(y_control);            // Empirical mean of y_control
  real<lower=0> sigma_control_observed = sd(y_control);           // Empirical standard deviation of y_control
  
  real<lower=0> mu_experiment_observed = mean(y_experiment);      /

In [16]:
model = CmdStanModel(stan_file='%s/%s.stan'%(name,name))

17:36:43 - cmdstanpy - INFO - compiling stan file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model4_pooled/model4_pooled.stan to exe file /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model4_pooled/model4_pooled
17:36:55 - cmdstanpy - INFO - compiled model executable: /Users/jasonmohabir/Documents/Duke/Courses/GraphicalModels/final_project/model4_pooled/model4_pooled


In [17]:
import pandas as pd

# Load the dataset
final_data = pd.read_csv("final/data1.tsv", sep='\t')

# Define the gene and enhancer IDs
geneID, enhancerID = 1, 2

# Filter the data for the specified gene
data_subset = final_data[final_data.geneID == geneID]

# Control samples
control_samples = data_subset[data_subset.enhancerID == -1]
N_control = control_samples.shape[0]

# Experimental samples for the specified enhancer
experiment_samples = data_subset[data_subset.enhancerID != -1]
N_experiment = experiment_samples.shape[0]

# Get the expression levels for control and experimental samples
y_control = control_samples.expression.values
y_experiment = experiment_samples.expression.values

# Map guides to indices and adjust for 1-based indexing in STAN
guide = experiment_samples.guideID.values
guide = guide - min(guide) + 1  # Adjusting guide indices

# Map enhancers to indices and adjust for 1-based indexing in STAN
enhancer = experiment_samples.enhancerID.values
enhancer = enhancer - min(enhancer) + 1  # Adjusting enhancer indices

# Define the total number of enhancers and guides
N_enhancers = len(data_subset.enhancerID.unique()) - 1
N_guides = len(data_subset.guideID.unique()) - 1

# Prepare data for CMDSTANPY
data = {
    'N_control': N_control,
    'N_experiment': N_experiment,
    'N_enhancers': N_enhancers,
    'N_guides': N_guides,
    'y_control': y_control,
    'y_experiment': y_experiment,
    'guide': guide,
    'enhancer': enhancer
}

data

{'N_control': 30,
 'N_experiment': 750,
 'N_enhancers': 5,
 'N_guides': 25,
 'y_control': array([25763, 19388, 17405, 29517, 17733, 19403, 15941, 25749, 27686,
        28957, 22363, 23709, 20466, 18504, 24131, 18546, 20252, 31070,
        24465, 18910, 19647, 23277, 21495, 19475, 22885, 13687, 25413,
        21491, 25463, 20526]),
 'y_experiment': array([12511, 20325, 17522, 18638, 17095, 17369, 16183, 18596, 20082,
        20636, 16309, 19589, 21810, 17050, 16087, 18701, 14388, 16330,
        22433, 18938, 17229, 16087, 18648, 13669, 21365, 17893, 17906,
        14929, 19881, 18415, 20236, 12146, 21895, 20240, 17056, 19924,
        15047, 19269, 17350, 21602, 27277, 14685, 19926, 14998, 20921,
        17879, 15242, 20382, 13826, 17104, 22051, 14873, 22124, 21276,
        21302, 16012, 20544, 18906, 17677, 20616, 22644, 20238, 19662,
        21772, 20429, 19062, 27283, 25803, 18689, 25038, 25207, 12985,
        25102, 10637, 16653, 30426, 19316, 23801, 20191, 19080, 20587,
        2030

In [18]:
print("=====Truth=====")

print('GeneID:',geneID)
print('Y_control expression:',round(y_control.mean(),2),round(y_control.std(),2))
print('Y_experiment expression:',round(y_experiment.mean(),2),round(y_experiment.std(),2))
print('Naive GeneID Beta Estimator:',round(y_experiment.mean()/y_control.mean(),4))

enhancer_guide_agg = data_subset.groupby(['enhancerID','guideID']).agg({'cellID':'count','expression':'mean'})
enhancer_guide_agg['Beta_Guide'] = enhancer_guide_agg['expression'].apply(lambda x: x / y_control.mean())
enhancer_guide_agg['Guide_Function_Prediction'] = enhancer_guide_agg['Beta_Guide'].apply(lambda x: 'Functional Guide [1]' if x < .92 else 'Non-Functional Guide [0]')

enhancer_grouped = enhancer_guide_agg.groupby('enhancerID').agg(list)

print('Sophisticated Beta Estimator:')
for ix,i in enhancer_grouped.iterrows():
    functional_guide_betas = [guide for status,guide in list(zip(i.Guide_Function_Prediction,i.Beta_Guide)) if not 'Non' in status]
    if ix > 0:
        print('\t\u03B2 EnhancerID %s:'%(ix),round(np.mean(functional_guide_betas),2))



=====Truth=====
GeneID: 1
Y_control expression: 22110.57 4080.39
Y_experiment expression: 17400.65 6320.54
Naive GeneID Beta Estimator: 0.787
Sophisticated Beta Estimator:
	β EnhancerID 1: 0.84
	β EnhancerID 2: 0.59
	β EnhancerID 3: 0.38
	β EnhancerID 4: 0.67
	β EnhancerID 5: 0.31


In [ ]:
model_fit = model.sample(data=data,thin=1,iter_sampling=1500,iter_warmup=50,seed=11051999,max_treedepth=100)


17:36:59 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

In [344]:
print(model_fit.diagnose())

Processing csv files: /var/folders/pm/gdtrt3dn5l19fhm45t9qtmdh0000gn/T/tmp0d_q4uwa/model4_pooled8o4wzvtx/model4_pooled-20241109160624_1.csv, /var/folders/pm/gdtrt3dn5l19fhm45t9qtmdh0000gn/T/tmp0d_q4uwa/model4_pooled8o4wzvtx/model4_pooled-20241109160624_2.csv, /var/folders/pm/gdtrt3dn5l19fhm45t9qtmdh0000gn/T/tmp0d_q4uwa/model4_pooled8o4wzvtx/model4_pooled-20241109160624_3.csv, /var/folders/pm/gdtrt3dn5l19fhm45t9qtmdh0000gn/T/tmp0d_q4uwa/model4_pooled8o4wzvtx/model4_pooled-20241109160624_4.csv

Checking sampler transitions treedepth.
Treedepth satisfactory for all transitions.

Checking sampler transitions for divergences.
No divergent transitions found.

Checking E-BFMI - sampler transitions HMC potential energy.
E-BFMI satisfactory.

Effective sample size satisfactory.

Split R-hat values satisfactory all parameters.

Processing complete, no problems detected.



In [ ]:
model_fit_mean = model_fit.summary().Mean.to_dict()

# print("=====Posterior Mean for Gene-Enhancer Betas=====") 

# # Initialize the nested dictionary
# nested_dict = {}

# # Iterate through the median_values dictionary
# for key, value in posterior_mean.items():
#     if '[' in key and ']' in key:
#         # Split the key to get the outer and inner keys
#         outer_key, inner_key = key.split('[')
#         inner_key = inner_key.rstrip(']')  # Remove the closing bracket
        
#         # Ensure the outer key exists in the nested dictionary
#         if outer_key not in nested_dict:
#             nested_dict[outer_key] = {}
        
#         # Assign the value to the appropriate place in the nested dictionary
#         nested_dict[outer_key][int(inner_key)] = value
#     else:
#         # If the key does not have an index, add it directly to the nested dictionary
#         nested_dict[key] = value

# posterior_mean = nested_dict

# { i for i in model_fit_mean.items() if 'B' in i[0]}


=====Posterior Mean for Gene-Enhancer Betas=====


{('B[1]', 0.784836),
 ('B[2]', 0.442248),
 ('B[3]', 0.285432),
 ('B[4]', 0.582298),
 ('B[5]', 0.255602),
 ('B[6]', 0.462382)}

In [416]:
draws = model_fit.draws_pd()
print("=====Posterior Mean for Gene-Enhancer Betas=====") 
posterior_mean = draws[[i for i in draws.columns if 'B' in i]].apply('mean').squeeze().to_dict()
print(posterior_mean)
print("=====Posterior Median for Gene-Enhancer Betas=====") 
posterior_median = draws[[i for i in draws.columns if 'B' in i]].apply('median').squeeze().to_dict()
print(posterior_median)
print("=====Posterior Mode for Gene-Enhancer Betas=====") 
posterior_mode = draws[[i for i in draws.columns if 'B' in i]].apply('mode').squeeze().to_dict()
print(posterior_mode)

=====Posterior Mean for Gene-Enhancer Betas=====
{'B[1]': 0.7848355505883334, 'B[2]': 0.4422481312035267, 'B[3]': 0.28543200221992165, 'B[4]': 0.5822980471946166, 'B[5]': 0.2556017814821834, 'B[6]': 0.46238169288473274}
=====Posterior Median for Gene-Enhancer Betas=====
{'B[1]': 0.8094224999999999, 'B[2]': 0.5206105000000001, 'B[3]': 0.3327345, 'B[4]': 0.6392085000000001, 'B[5]': 0.284158, 'B[6]': 0.42173550000000004}
=====Posterior Mode for Gene-Enhancer Betas=====
{'B[1]': 0.796607, 'B[2]': 0.562009, 'B[3]': 0.20731, 'B[4]': 0.659516, 'B[5]': 0.279763, 'B[6]': 0.833153}


In [4]:
true_betas = [0.88, 0.59, 0.39, 0.7, 0.32]

print("=====True Gene-Enhancer Betas=====")
for i in enumerate(true_betas):
    print('EnhancerID %s:'%(i[0]+1),i[1])


=====True Gene-Enhancer Betas=====
EnhancerID 1: 0.88
EnhancerID 2: 0.59
EnhancerID 3: 0.39
EnhancerID 4: 0.7
EnhancerID 5: 0.32


In [12]:
final_data.geneID.nunique()


1000